In [1]:
# Some imports, we are not gong to use all the imports in this workbook but in subsequent workbooks we surely will.
import os
import time
import urllib
import zipfile
import time
#from google.colab import files

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics
import matplotlib.pyplot as plt
import seaborn
from toolz import pipe as p
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from tensorflow.keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras.engine.topology import Layer
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *


Using TensorFlow backend.


# New Section

In [6]:
# Define some Global Variables
max_features = 20000 # Maximum Number of words we want to include in our dictionary
maxlen = 400 # No of words in question we want to create a sequence with
embed_size = 50# Size of word to vec embedding we are using


# Loading the data
def load_and_prec(sample_size):
    #data = pd.read_csv("pubmed_cr_hep_ctl_abstracts_clean.csv")

    hep_pos = x[x['label_hep'] == 1].sample(sample_size)
    hep_neg = x[x['label_hep'] == 0].sample(sample_size)


    combined = pd.concat([hep_pos, hep_neg])

    X = combined['txt']
    y = combined['label_hep']

    print(len(X))
    hep_pos = combined[combined['label_hep'] == 1]
    hep_neg = combined[combined['label_hep'] == 0]
    print(len(hep_neg))
    print(len(hep_pos))

    print("X shape : ", X.shape)
    print("y shape : ", y.shape)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

    ## split to train and val
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)  # .08 since the datasize is large enough.

    ## Tokenize the sentences
    '''
    keras.preprocessing.text.Tokenizer tokenizes(splits) the texts into tokens(words).
    Signature:
    Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
    lower=True, split=' ', char_level=False, oov_token=None, document_count=0, **kwargs)

    The num_words parameter keeps a prespecified number of words in the text only. 
    It also filters some non wanted tokens by default and converts the text into lowercase.

    It keeps an index of words(dictionary of words which we can use to assign a unique number to a word) 
    which can be accessed by tokenizer.word_index.
    For example - For a text corpus the tokenizer word index might look like. 
    The words in the indexed dictionary are sort of ranked in order of frequencies,
    {'the': 1,'what': 2,'is': 3, 'a': 4, 'to': 5, 'in': 6, 'of': 7, 'i': 8, 'how': 9}

    The texts_to_sequence function converts every word(token) to its respective index in the word_index

    So Lets say we started with 
    train_X as something like ['This is a sentence','This is another bigger sentence']
    and after fitting our tokenizer we get the word_index as {'this':1,'is':2,'sentence':3,'a':4,'another':5,'bigger':6}
    The texts_to_sequence function will tokenize the sentences and replace words with individual tokens to give us 
    train_X = [[1,2,4,3],[1,2,5,6,3]]
    '''
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(x_train))
    x_train = tokenizer.texts_to_sequences(x_train)
    x_val = tokenizer.texts_to_sequences(x_val)
    x_test = tokenizer.texts_to_sequences(x_test)

    ## Pad the sentences. We need to pad the sequence with 0's to achieve consistent length across examples.
    '''
    We had train_X = [[1,2,4,3],[1,2,5,6,3]]
    lets say maxlen=6
        We will then get 
        train_X = [[1,2,4,3,0,0],[1,2,5,6,3,0]]
    '''
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_val = pad_sequences(x_val, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)

    return x_train, x_val, x_test, y_train, y_val, y_test, tokenizer.word_index

# https://www.kaggle.com/yekenot/2dcnn-textclassifier
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, trainable=True)(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
    64*70(maxlen)*2(bidirection concat)
    '''
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# This defines the HAN (an lstm with attention)
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, trainable=True)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# This creates a CNN model with random embeddings
def cnn_model(sequence_length, filter_sizes, EMBEDDING_DIM, embed_layer, num_filters=0, drop_rate=0):
    inputs = Input(shape=(maxlen,))
    embedding = Embedding(max_features, embed_size)(inputs)
    reshape = Reshape((maxlen, embed_size, 1))(embedding)

    conv_0 = Conv2D(num_filters, (filter_sizes[0], embed_size), activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv_1 = Conv2D(num_filters, (filter_sizes[1], embed_size), activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv_2 = Conv2D(num_filters, (filter_sizes[2], embed_size), activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(reshape)

    maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1, 1))(conv_0)
    maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1))(conv_2)

    merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
    flatten = Flatten()(merged_tensor)
    reshape = Reshape((3 * num_filters,))(flatten)
    dropout = Dropout(drop_rate)(reshape)
    output = Dense(1, activation="sigmoid")(dropout)

    model = Model(inputs, output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_pred(model, x_train, y_train, x_val, y_val, epochs):
    # can go down to
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.0001, verbose=2)
    # Patients make it 5
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=2, mode='auto')
    callbacks = [reduce_lr, earlystopping]
    model.fit(x_train, y_train, batch_size=512, epochs=epochs, validation_data=(x_val, y_val),callbacks=callbacks)
    pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def get_config(self):
      config = super().get_config().copy()
      config.update({
              'W_regularizer': self.W_regularizer,
              'u_regularizer': self.u_regularizer,
              'b_regularizer': self.b_regularizer,
              'W_constraint': self.W_constraint,
              'u_constraint': self.u_constraint,
              'b_constraint': self.b_constraint,
              'bias': self.bias,
      })
      return config

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

def focal_loss(gamma=2, alpha=0.75):
    def focal_loss_fixed(y_true, y_pred):#with tensorflow
        eps = 1e-12
        y_pred=K.clip(y_pred,eps,1.-eps)#improve the stability of the focal loss and see issues 1 for more information
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed


# Training method for the LSTM
def train_pred_lstm(model, epochs=2):
    filepath="weights_best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.0001, verbose=2)
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=2, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y),callbacks=callbacks)
    model.load_weights(filepath)
    pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

# Computes the mean confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Manual mean method (could also use math.mean)
def mean(l):
    return sum(l)/float(len(l))


# This is the main logging funciton that lets youo check on the results from
# each iteration
def log_overall_output(j, lstm_f1, lstm_roc, han_f1, han_roc, cnn_f1, cnn_roc, cnn_avg_runtime, lstm_avg_runtime, han_avg_runtime):

    f = open("output2/" + str(j) + "/test.txt", "w")
    f.close()
    np.save("output2/" + str(j) + "/" + "lstm_f1", lstm_f1)
    np.save("output2/" + str(j) + "/" + "lstm_roc", lstm_roc)
    np.save("output2/" + str(j) + "/" + "han_f1", han_f1)
    np.save("output2/" + str(j) + "/" + "han_roc", han_roc)
    np.save("output2/" + str(j) + "/" + "cnn_f1", cnn_f1)
    np.save("output2/" + str(j) + "/" + "cnn_roc", cnn_roc)


    log_overall = open("output2/" + str(j) + "/output_overall.txt", "w")
    log_overall.write('Sample Size: ' + str(j) + '\n\n')
    log_overall.write('LSTM Average F1: ' + str(mean(lstm_f1)) + '\n')
    log_overall.write('LSTM STANDARD DEVIATION F1: ' + str(statistics.stdev(lstm_f1)) + '\n')
    log_overall.write('LSTM F1 Confidence Interval: ' + str(mean_confidence_interval(lstm_f1)) + '\n')
    log_overall.write('LSTM Average AUC: ' + str(mean(lstm_roc)) + '\n')
    log_overall.write('LSTM STANDARD DEVIATION AUC: ' + str(statistics.stdev(lstm_roc)) + '\n')
    log_overall.write('LSTM AUC Confidence Interval: ' + str(mean_confidence_interval(lstm_roc)) + '\n')
    log_overall.write('LSTM Average Runtime: ' + str(lstm_avg_runtime)+ '\n\n')

    log_overall.write('HAN Average F1: ' + str(mean(han_f1)) + '\n')
    log_overall.write('HAN STANDARD DEVIATION F1: ' + str(statistics.stdev(han_f1)) + '\n')
    log_overall.write('HAN F1 Confidence Interval: ' + str(mean_confidence_interval(han_f1)) + '\n')
    log_overall.write('HAN Average AUC: ' + str(mean(han_roc)) + '\n')
    log_overall.write('HAN STANDARD DEVIATION AUC: ' + str(statistics.stdev(han_roc)) + '\n')
    log_overall.write('HAN AUC Confidence Interval: ' + str(mean_confidence_interval(han_roc)) + '\n')
    log_overall.write('HAN Average Runtime: ' + str(han_avg_runtime)+ '\n\n')

    log_overall.write('CNN Average F1: ' + str(mean(cnn_f1)) + '\n')
    log_overall.write('CNN STANDARD DEVIATION F1: ' + str(statistics.stdev(cnn_f1)) + '\n')
    log_overall.write('CNN F1 Confidence Interval: ' + str(mean_confidence_interval(cnn_f1)) + '\n')
    log_overall.write('CNN Average AUC: ' + str(mean(cnn_roc)) + '\n')
    log_overall.write('CNN STANDARD DEVIATION AUC: ' + str(statistics.stdev(cnn_roc)) + '\n')
    log_overall.write('CNN AUC Confidence Interval: ' + str(mean_confidence_interval(cnn_roc)) + '\n')
    log_overall.write('CNN Average Runtime: ' + str(cnn_avg_runtime)+ '\n\n')


    log_overall.write(
        'p-value of t-test for CNN AUC vs LSTM AUC: ' + str(scipy.stats.ttest_ind(cnn_roc, lstm_roc, equal_var=False)) + '\n')
    log_overall.write(
        'p-value of t-test for CNN AUC vs HAN AUC: ' + str(scipy.stats.ttest_ind(cnn_roc, han_roc, equal_var=False)) + '\n')
    log_overall.write(
        'p-value of t-test for LSTM AUC vs HAN AUC: ' + str(
            scipy.stats.ttest_ind(lstm_roc, han_roc, equal_var=False)) + '\n')

    log_overall.write(
        'p-value of t-test for CNN F1 vs LSTM F1: ' + str(scipy.stats.ttest_ind(cnn_f1, lstm_f1, equal_var=False)) + '\n')
    log_overall.write(
        'p-value of t-test for CNN F1 vs HAN F1: ' + str(scipy.stats.ttest_ind(cnn_f1, han_f1, equal_var=False)) + '\n')
    log_overall.write(
        'p-value of t-test for LSTM F1 vs HAN F1: ' + str(
            scipy.stats.ttest_ind(lstm_f1, han_f1, equal_var=False)) + '\n')
    log_overall.close()


def log_iterations(filename, f1, roc_auc, run_time):
    f = open(filename, "w")
    f.write('F1: {}\n'.format(f1))
    f.write('ROC AUC: {}\n'.format(roc_auc))
    f.write('Time to Train: ' + str(run_time) + ' seconds')
    f.close()

# This function plots the overall trend of f1 and ROC AUC score
def plot_function(statistic, dictionary):
    global key, fig1, ax1
    for key in dictionary.keys():
        fig1, ax1 = plt.subplots()
        ax1.set_title(str(key).upper() + " " + statistic + " vs Number of Records")
        ax1.set_ylabel(statistic + " Scores")
        ax1.set_xlabel("Number of Records")
        ax1.plot(y_axis, roc_dict[key], label=key)
        plt.savefig("output2/" + str(key) + "_" + statistic + "_linechart.png")
        plt.clf()

# This function is for pllotting the bxplots for each of the models
def plot_boxplot(statistic, output_dict):
    global fig1, ax1
    fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, sharex=True)
    fig1.subplots_adjust(hspace=0.3)
    fig1.suptitle('Boxplots of ' + statistic)
    ax1.set_title(output_dict[0].get('name'))
    ax1.boxplot([output_dict[0].get('lstm'), output_dict[0].get('han'), output_dict[0].get('cnn')], labels=["LSTM", "HAN", "CNN"])
    ax2.set_title(output_dict[1].get('name'))
    ax2.boxplot([output_dict[1].get('lstm'), output_dict[1].get('han'), output_dict[1].get('cnn')], labels=["LSTM", "HAN", "CNN"])
    ax3.set_title(output_dict[2].get('name'))
    ax3.boxplot([output_dict[2].get('lstm'), output_dict[2].get('han'), output_dict[2].get('cnn')], labels=["LSTM", "HAN", "CNN"])
    ax4.set_title(output_dict[3].get('name'))
    ax4.boxplot([output_dict[3].get('lstm'), output_dict[3].get('han'), output_dict[3].get('cnn')], labels=["LSTM", "HAN", "CNN"])
    plt.savefig("output2/" + statistic + "_overall_boxplot.png")
    plt.clf()

def run_model_testing(model, model_name, iteration):
    ## Train the Model
    start_time = time.perf_counter()
    pred_val_y, pred_test_y = train_pred(model, train_X, train_y,
                                          val_X, val_y, epochs=50)
    end_time = time.perf_counter()
    one_train_time = end_time - start_time

    f1 = metrics.f1_score(test_y, (pred_test_y > 0.50).astype(int))
    roc_auc = metrics.roc_auc_score(test_y, pred_test_y)
    pred_test_y = (pred_test_y > 0.5).astype(int)
    # Log the Model
    filename = "output2/"+ str(j) + "/output_" + model_name + str(iteration) + ".txt"
    log_iterations(filename, f1, roc_auc, one_train_time)
    return f1, roc_auc, one_train_time

def plot_seaborn_boxplot(data):
  fig, ax = plt.subplots()
  fig2, ax2 = plt.subplots
  ax.set_title("Boxplot of AUC Scores")
  plot = seaborn.boxplot(data=data, x='Sample Size', y='AUC', hue='Model', ax=ax)
  ax.get_figure().savefig('output2/ax_auc.png')
  ax2.set_title("Boxplot of F1 Scores")
  plot2 = seaborn.boxplot(data=data, x='Sample Size', y='F1', hue='Model', ax=ax2)
  ax2.get_figure().savefig('output2/ax_f1.png')
  
  



# Check functions

In [7]:
x = pd.read_csv('file_50_epochs_final/pubmed_cr_hep_ctl_abstracts_clean.csv')

In [17]:
x_train, x_val, x_test, y_train, y_val, y_test, word_index = load_and_prec(500)

1000
500
500
X shape :  (1000,)
y shape :  (1000,)


In [13]:
x_test.shape

(330, 400)

In [14]:
x_train.shape

(603, 400)

In [15]:
x_val.shape

(67, 400)

In [16]:
list(range(500, 2500, 500))

[500, 1000, 1500, 2000]

In [29]:
sizes = [{'size': _, 'train_shape': load_and_prec(_)[0].shape, 'test_shape': load_and_prec(_)[1].shape, 'val_shape': load_and_prec(_)[2].shape
     } for _ in range(500, 2500, 500)]

1000
500
500
X shape :  (1000,)
y shape :  (1000,)
1000
500
500
X shape :  (1000,)
y shape :  (1000,)
1000
500
500
X shape :  (1000,)
y shape :  (1000,)
2000
1000
1000
X shape :  (2000,)
y shape :  (2000,)
2000
1000
1000
X shape :  (2000,)
y shape :  (2000,)
2000
1000
1000
X shape :  (2000,)
y shape :  (2000,)
3000
1500
1500
X shape :  (3000,)
y shape :  (3000,)
3000
1500
1500
X shape :  (3000,)
y shape :  (3000,)
3000
1500
1500
X shape :  (3000,)
y shape :  (3000,)
4000
2000
2000
X shape :  (4000,)
y shape :  (4000,)
4000
2000
2000
X shape :  (4000,)
y shape :  (4000,)
4000
2000
2000
X shape :  (4000,)
y shape :  (4000,)


In [30]:
sizes

[{'size': 500,
  'train_shape': (603, 400),
  'test_shape': (67, 400),
  'val_shape': (330, 400)},
 {'size': 1000,
  'train_shape': (1206, 400),
  'test_shape': (134, 400),
  'val_shape': (660, 400)},
 {'size': 1500,
  'train_shape': (1809, 400),
  'test_shape': (201, 400),
  'val_shape': (990, 400)},
 {'size': 2000,
  'train_shape': (2412, 400),
  'test_shape': (268, 400),
  'val_shape': (1320, 400)}]

In [ ]:
if __name__ == '__main__':
  with tf.device('/device:GPU:0'):
      x = pd.read_csv('/content/pubmed_cr_hep_ctl_abstracts_clean.csv')
      hep_pos = x[x['label_hep'] == 1]
      hep_neg = x[x['label_hep'] == 0]

      print(len(hep_neg))
      print(len(hep_pos))

      lstm_f1 = []
      lstm_roc = []
      han_f1 = []
      han_roc = []
      cnn_f1 = []
      cnn_roc = []

      auc_array = []
      f1_array = []

      model = []
      sample_size = []
      f1_arr = []
      auc_arr = []

      cnn_run_time = []
      lstm_run_time = []
      han_run_time = []

      roc_dict = {
          'lstm': [],
          'han': [],
          'cnn': []
      }
      f1_dict = {
          'lstm': [],
          'han': [],
          'cnn': []
      }
      y_axis = []

      for j in range(500, 2500, 500):
          #density plot
          if not os.path.exists('output2/' + str(j)):
              os.makedirs('output2/' + str(j))
          for i in range(0, 25):
              train_X, val_X, test_X, train_y, val_y, test_y, word_index = load_and_prec(j)
              
              print(test_X.shape[1])
              cnn_shape = test_X.shape[1]

              # CNN Testing
              
              cnn_model_built = cnn_model(cnn_shape, [3, 4, 5], 200, [], num_filters=200, drop_rate=0.2)
              cnn_model_built.summary()
             
              f1, roc, run_time = run_model_testing(cnn_model_built, 'cnn', i)
              cnn_f1.append(f1)
              cnn_roc.append(roc)
              cnn_run_time.append(run_time)
              model.append('CNN')
              sample_size.append(j)
              f1_arr.append(f1)
              auc_arr.append(roc)

              # # LSTM Testing
              lstm_model = model_lstm_du([])
              lstm_model.summary()
         

              f1, roc, run_time = run_model_testing(lstm_model, 'lstm', i)
              lstm_f1.append(f1)
              lstm_roc.append(roc)
              lstm_run_time.append(run_time)
              model.append('LSTM')
              sample_size.append(j)
              f1_arr.append(f1)
              auc_arr.append(roc)

              # HAN Testing
              han_model = model_lstm_atten([])
              han_model.summary()
             

              f1, roc, run_time = run_model_testing(han_model, 'han', i)
              han_f1.append(f1)
              han_roc.append(roc)
              han_run_time.append(run_time)
              model.append('HAN')
              sample_size.append(j)
              f1_arr.append(f1)
              auc_arr.append(roc)

          log_overall_output(j, lstm_f1, lstm_roc, han_f1, han_roc, cnn_f1, cnn_roc, mean(cnn_run_time), mean(lstm_run_time), mean(han_run_time))
          f1_array.append({
              'name': j,
              'lstm': lstm_f1,
              'han': han_f1,
              'cnn': cnn_f1
          })
          auc_array.append({
              'name': j,
              'lstm': lstm_roc,
              'han': han_roc,
              'cnn': cnn_roc
          })
          
          roc_dict["lstm"].append(mean(lstm_roc))
          roc_dict["han"].append(mean(han_roc))
          roc_dict["cnn"].append(mean(cnn_roc))
          f1_dict["lstm"].append(mean(lstm_f1))
          f1_dict["han"].append(mean(han_f1))
          f1_dict["cnn"].append(mean(cnn_f1))
          y_axis.append(j)
      data = pd.DataFrame({
          'Model': model,
          'Sample Size': sample_size,
          'F1': f1_arr,
          'AUC': auc_arr
      })
      data.to_excel("output" + str(j) + ".xlsx") 
      plot_boxplot("F1", f1_array)
      plot_boxplot("AUC", auc_array)
      plot_function("AUC", roc_dict)
      plot_function("F1", f1_dict)
!zip -r /content/file_50_epochs.zip /content/output2
files.download('/content/file_50_epochs.zip')
# files.download('/content/output.xlsx')
